# Running Circuits on Hardware

This tutorial shows how a quantum circuit can be run on actual quantum hardware.

## Import all Necessary Libraries

In [1]:
from NoisyCircuits.QuantumCircuit import QuantumCircuit as nqc
from NoisyCircuits.utils.GetNoiseModel import GetNoiseModel
from NoisyCircuits.RunOnHardware import RunOnHardware
import os
import matplotlib.pyplot as plt

2025-11-21 18:24:41,809	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Define the Necessary Parameters

In [ ]:
token = os.environ["ibm_api_key"] # Replace with your IBM Quantum API token
backend_name = "ibm_fez"
shots = 1024
verbose = False
jsonize = True
num_qubits = 2
num_trajectories = 100
qpu = "heron" # Only possible option for IBM backends at the moment
num_cores = 4

## Get the Noise Model

In [3]:
noise_model = GetNoiseModel(
    token=token,
    backend_name=backend_name
).get_noise_model()

qiskit_runtime_service._discover_account:WARNING:2025-11-21 18:24:47,786: Loading account with the given token. A saved account will not be used.
qiskit_runtime_service.__init__:WARNING:2025-11-21 18:24:50,849: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: Open_Sys. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-11-21 18:24:50,853: Using instance: Open_Sys, plan: open
qiskit_runtime_service.backends:WARNING:2025-11-21 18:24:51,043: Using instance: Open_Sys, plan: open


## Build the Quantum Circuit

For this tutorial, we stick to the simple Bell State circuit

In [4]:
circuit = nqc(
    num_qubits=num_qubits,
    noise_model=noise_model,
    backend_qpu_type=qpu,
    num_trajectories=num_trajectories,
    num_cores=num_cores,
    jsonize=jsonize,
    verbose=verbose
)

2025-11-21 18:25:04,318	INFO worker.py:2012 -- Started a local Ray instance.
/Users/adam-ukj7r05xnu2fywx/miniconda3/envs/NoisyCircuits_New_Branch/lib/python3.10/site-packages/ray/_private/worker.py:2051: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(


In [5]:
circuit.refresh()
circuit.H(0)
circuit.CX(0, 1)

## Perform a Simulation

This is so that we can compare the results of a simulated quantum circuit to actual results from hardware

In [6]:
probs_simulated = circuit.execute(qubits=[0, 1], num_trajectories=100)

## Initialize the Hardware Interface Module

In [7]:
hardware_runner = RunOnHardware(
    token=token,
    backend=backend_name,
    shots=shots
)

qiskit_runtime_service._discover_account:WARNING:2025-11-21 18:25:11,686: Loading account with the given token. A saved account will not be used.
qiskit_runtime_service.__init__:WARNING:2025-11-21 18:25:14,384: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: Open_Sys. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-11-21 18:25:14,386: Loading instance: Open_Sys, plan: open


## Submitting the Circuit

Create the low-level circuit instructions for the hardware

In [8]:
hardware_runner.create_circuits(
    circuit=circuit,
    measure_qubits=[0, 1])

TypeError: Value after * must be an iterable, not float

[2025-11-21 18:25:34,836 E 250231 250609] core_worker_process.cc:825: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14


Compile the circuit for submission

In [ ]:
hardware_runner.setup_circuits()

Submit the circuit to IBM hardware. This returns the unique job id from IBM that can be used to retreive the results.

In [ ]:
job_id = hardware_runner.run()

## Retrieve Results from Hardware

In [ ]:
results_from_hardware = hardware_runner.get_results(job_id=job_id)
probs_hardware = results_from_hardware[0]

## Compare Results

In [ ]:
plt.figure(figsize=(12, 5))
x_labels = [r"$|{}{}\rangle$".format(i, j) for i in range(2) for j in range(2)]

plt.bar(x_labels, probs_simulated, width=0.4, label="Simulated", alpha=0.7, color="C0", align="edge")
plt.bar(x_labels, probs_hardware, width=-0.4, label="Hardware", alpha=0.7, color="C1", align="edge")
plt.xlabel("States", fontsize=14)
plt.ylabel("Probabilities", fontsize=14)
plt.title("Comparison of Simulated vs Hardware Results", fontsize=16)
plt.legend(fontsize=12)
plt.grid(axis="y")
plt.show()

## Shutdown

In [ ]:
circuit.shutdown()